**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [17]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Reshape, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act allows the agent to interact with the environment and take an action to move to the next state. In this case we have four actions which are randomly chosen and if the agent chooses a seemingly good action on chance then it may lock on to it even if it is suboptimal and keep exploiting the action without exploring the other options. To explore other options we introduce a parameter called epsilon which is a number between 0 and 1, and intoduce a condition that if the random number genrerated is less than epsilon then we stick to the current best action else we explore other action-states. Therfore the probability of exploring new action-states is given by the value of epsilon.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

position: it gives the current location of the rat on the island, with value set to 1

board: it is the entire area of the island defined by (gridsize * gridsize), each cell contains a potential reward based on the action of the rat. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0 ,4,size=1)
        pass

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [9]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
         # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.0/11.0. Average score (-3.0)
Win/lose count 11.0/17.0. Average score (-4.5)
Win/lose count 10.0/16.0. Average score (-5.0)
Win/lose count 10.5/14.0. Average score (-4.625)
Win/lose count 9.0/11.0. Average score (-4.1)
Win/lose count 12.5/14.0. Average score (-3.6666666666666665)
Win/lose count 15.5/15.0. Average score (-3.0714285714285716)
Win/lose count 13.0/12.0. Average score (-2.5625)
Win/lose count 11.0/15.0. Average score (-2.7222222222222223)
Win/lose count 11.5/12.0. Average score (-2.5)
Win/lose count 8.5/22.0. Average score (-3.5)
Win/lose count 6.0/12.0. Average score (-3.7083333333333335)
Win/lose count 10.5/11.0. Average score (-3.4615384615384617)
Win/lose count 11.5/10.0. Average score (-3.107142857142857)
Win/lose count 8.5/8.0. Average score (-2.8666666666666667)
Win/lose count 7.5/10.0. Average score (-2.84375)
Win/lose count 11.5/17.0. Average score (-3.0)
Win/lose count 7.0/11.0. Average score (-3.0555555555555554)
Win/lose count 11.0/15.0. Average 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




(1)

Given the Bellman equation:

$$\begin{align*}
Q^{\pi}(s,a) &amp; = E_{\pi}[\Sigma^{\infty}_{t=0} \gamma^t r_t(s_t, \pi(s_t),s_{t+1}) | s_t = s,a_t = a] \\
\Rightarrow Q^{\pi}(s_0,a_0) &amp; = E_{\pi}[r_0(s_0,a_0,s_1)+\Sigma^{\infty}_{t=1} \gamma^t r_t (s_t,\pi(s_t),s_{t+1}) | s_0,a_0 ] \\
&amp; = E_{\pi}[r_0(s_0,a_0,s_1) + \gamma E_{\pi} \{ \Sigma_{t=1}^{\infty} \gamma^{t-1} r_t (s_t,\pi(s_t),s_{t+1} | s_1,a_1 \} s_0,a_0 ] \\
&amp; = E_{\pi} [r_0 (s_0,a_0,s_1)+\gamma Q^{\pi}(s_1,a_1)]
\end{align*}$$
Therefore: $${Q^{\pi}(s,a) = E_{(s',a') \sim p(.|s,a)} [r(s,a)+\gamma Q^{\pi}(s',a')]}$$(2)

Given the Bellman equation:

$$V^{\pi}(s_0) = E[\Sigma^{\infty}_{t=0} \gamma^t r_t (s_t,\pi(s_t),s_{t+1} | s_0] $$
and $$V^*(s_0) = \max E[\Sigma^{\infty}_{t=0}\gamma^t r_t (s_t,\pi(s_t),s_{t+1} | s_0] $$

Then

$$\begin{align*} V^*(s_0) &amp; = \underset{\pi,a_0}{\max} E[r_0(s_0,a_0,s_1) + \Sigma^{\infty}_{t=1} \gamma^t r_t (s_t,\pi(s_t),s_(t+1)|s_0] \\
&amp; = \underset{a_0}{\max} E_{s_1} [r_0(s_0,a_0,s_1)+ \gamma \underset{\pi}{\max} \Sigma^{\infty}_{t=1} E_{s_2,s_3...} \{ \Sigma^{\infty}_{t=1} \gamma^{t-1} r_t(s_t,\pi(s_{t+1}),s_{t+1} \}  | s_0] \\
&amp; = \underset{a_0}{\max} E_{s_1}[r_0(s_0,a_0,s_1)+\gamma V^* (s_1)] 
\end{align*}$$
So: $$ V^*(s) = \max E_{s'}[r(s,a,s')+\gamma V^*(s')|s] $$

Since $V^*(s) = \underset{a'}{\max}Q^*(s,a')$

$$\begin{align*}
\Rightarrow V^*(s) &amp; = \underset{a}{\max}E_{s'}[r(s,a,s')+\gamma \underset{a'}{\max} Q^*(s',a')|s,a] \\
\Rightarrow Q^*(s,a) &amp; = E_{s'}[r(s,a,s')+\gamma \underset{a'}{\max} Q^*(s',a')|s,a]
\end{align*}$$$$
{Q^*(s,a) = E_{s'\sim\pi^*(.|s,a)}[r(s,a)+\gamma \underset{a'}{\max} Q^*(s',a')]}$$
(3)

Given the loss function $\mathcal{L}(\theta)= [Q^*(s,a,\theta) - Q(s,a,\theta)]^2$

By substituting $Q^*(s,a,\theta)$, we get:

$$\mathcal{L}(\theta) = [E_{s'\sim \pi^*(.|s,a,s')}[r(s,a,s')+\gamma\max_{a'}Q^{*}(s',a,\theta)] - Q(s,a,\theta)]^2$$$$ \Rightarrow {\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}} $$


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        pass

    def random_access(self):
        r= np.random.randint(0, len(self.memory))
        return self.memory[r]

***
The pipeline we will use for training is given below:

In [12]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        acts_ = self.model.predict(np.reshape(s,(1,5,5,self.n_state)))
        return np.argmax(acts_[0])
        
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            current_state, next_state, action, reward, game_over_ = self.memory.random_access() #sample some moves
            input_states[i] = current_state
            
            s_reshape=np.reshape(current_state,(1,5,5,self.n_state))
            next_state=np.reshape(next_state,(1,5,5,self.n_state))
            target_q[i] = self.model.predict(s_reshape)
            
           
            if game_over_:

                target_q[i,a_]= reward
            else:
                q = self.model.predict(next_state)
                target_q[i,a_] = reward + self.discount * np.max(q)
        
        
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Reshape((5*5*self.n_state,), input_shape=(5,5,self.n_state)))
        model.add(Dense(128,input_shape=(50,),init='uniform',activation='relu'))
        model.add(Dense(64,init='uniform',activation='relu'))
        model.add(Dense(32,init='uniform',activation='relu'))
        model.add(Dense(16,activation='relu'))
        model.add(Dense(8,activation='relu'))
        model.add(Dense(4,init='uniform',activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Instructions for updating:
Colocations handled automatically by placer.


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_shape=(50,), activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="linear", kernel_initializer="uniform")`


Instructions for updating:
Use tf.cast instead.
Epoch 000/030 | Loss 0.0000 | Win/lose count 2.0/0 (2.0)
Epoch 001/030 | Loss 0.0019 | Win/lose count 1.0/4.0 (-3.0)
Epoch 002/030 | Loss 0.0040 | Win/lose count 4.5/2.0 (2.5)
Epoch 003/030 | Loss 0.0000 | Win/lose count 0.5/2.0 (-1.5)
Epoch 004/030 | Loss 0.0078 | Win/lose count 4.0/4.0 (0.0)
Epoch 005/030 | Loss 0.0019 | Win/lose count 1.5/1.0 (0.5)
Epoch 006/030 | Loss 0.0020 | Win/lose count 2.0/2.0 (0.0)
Epoch 007/030 | Loss 0.0000 | Win/lose count 4.5/6.0 (-1.5)
Epoch 008/030 | Loss 0.0000 | Win/lose count 4.0/7.0 (-3.0)
Epoch 009/030 | Loss 0.0098 | Win/lose count 6.5/7.0 (-0.5)
Epoch 010/030 | Loss 0.0273 | Win/lose count 8.0/12.0 (-4.0)
Epoch 011/030 | Loss 0.0098 | Win/lose count 16.5/13.0 (3.5)
Epoch 012/030 | Loss 0.0039 | Win/lose count 5.0/13.0 (-8.0)
Epoch 013/030 | Loss 0.0077 | Win/lose count 7.5/14.0 (-6.5)
Epoch 014/030 | Loss 0.0098 | Win/lose count 8.0/16.0 (-8.0)
Epoch 015/030 | Loss 0.0137 | Win/lose count 7.5/16.0 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32,kernel_size=(2,2),strides=(1,1),
                         activation='relu',input_shape=(5,5,self.n_state)))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Conv2D(64, (1, 1), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(4, activation='linear'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [21]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/030 | Loss 0.0307 | Win/lose count 4.0/5.0 (-1.0)
Epoch 001/030 | Loss 0.0011 | Win/lose count 5.5/3.0 (2.5)
Epoch 002/030 | Loss 0.0316 | Win/lose count 4.5/2.0 (2.5)
Epoch 003/030 | Loss 0.0066 | Win/lose count 4.0/10.0 (-6.0)
Epoch 004/030 | Loss 0.0140 | Win/lose count 4.5/8.0 (-3.5)
Epoch 005/030 | Loss 0.0105 | Win/lose count 5.5/14.0 (-8.5)
Epoch 006/030 | Loss 0.0161 | Win/lose count 6.5/9.0 (-2.5)
Epoch 007/030 | Loss 0.0196 | Win/lose count 3.5/4.0 (-0.5)
Epoch 008/030 | Loss 0.0099 | Win/lose count 3.5/2.0 (1.5)
Epoch 009/030 | Loss 0.0251 | Win/lose count 8.0/11.0 (-3.0)
Epoch 010/030 | Loss 0.0212 | Win/lose count 7.0/10.0 (-3.0)
Epoch 011/030 | Loss 0.0053 | Win/lose count 4.0/9.0 (-5.0)
Epoch 012/030 | Loss 0.0111 | Win/lose count 3.5/8.0 (-4.5)
Epoch 013/030 | Loss 0.0088 | Win/lose count 10.5/19.0 (-8.5)
Epoch 014/030 | Loss 0.0034 | Win/lose count 2.0/1.0 (1.0)
Epoch 015/030 | Loss 0.0209 | Win/lose count 4.5/11.0 (-6.5)
Epoch 016/030 | Loss 0.0177 | Win/los

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

# Temperature set to 0.3

In [24]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_shape=(50,), activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="linear", kernel_initializer="uniform")`


Test of the CNN
Win/lose count 1.5/1.0. Average score (0.5)
Win/lose count 1.0/2.0. Average score (-0.25)
Win/lose count 0.5/1.0. Average score (-0.3333333333333333)
Win/lose count 3.0/2.0. Average score (0.0)
Win/lose count 1.5/1.0. Average score (0.1)
Win/lose count 1.5/3.0. Average score (-0.16666666666666666)
Win/lose count 1.5/3.0. Average score (-0.35714285714285715)
Win/lose count 2.0/1.0. Average score (-0.1875)
Win/lose count 0.5/1.0. Average score (-0.2222222222222222)
Win/lose count 2.0/5.0. Average score (-0.5)
Win/lose count 2.5/5.0. Average score (-0.6818181818181818)
Win/lose count 1.0/5.0. Average score (-0.9583333333333334)
Win/lose count 2.0/2.0. Average score (-0.8846153846153846)
Win/lose count 2.0/4.0. Average score (-0.9642857142857143)
Win/lose count 1.0/0. Average score (-0.8333333333333334)
Win/lose count 4.0/3.0. Average score (-0.71875)
Win/lose count 2.5/6.0. Average score (-0.8823529411764706)
Win/lose count 2.5/2.0. Average score (-0.8055555555555556)
Win/

In [25]:
HTML(display_videos('cnn_test10.mp4'))

In [26]:
HTML(display_videos('fc_test10.mp4'))

# Temperature set to 0.9

In [35]:
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test_09_')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test_09_')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_shape=(50,), activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:80: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="uniform")`
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="linear", kernel_initializer="uniform")`


Test of the CNN
Win/lose count 5.0/0. Average score (5.0)
Win/lose count 4.5/3.0. Average score (3.25)
Win/lose count 4.0/0. Average score (3.5)
Win/lose count 10.0/2.0. Average score (4.625)
Win/lose count 7.0/1.0. Average score (4.9)
Win/lose count 6.0/2.0. Average score (4.75)
Win/lose count 8.5/0. Average score (5.285714285714286)
Win/lose count 7.5/1.0. Average score (5.4375)
Win/lose count 7.5/0. Average score (5.666666666666667)
Win/lose count 9.5/1.0. Average score (5.95)
Win/lose count 5.0/3.0. Average score (5.590909090909091)
Win/lose count 6.0/1.0. Average score (5.541666666666667)
Win/lose count 5.5/2.0. Average score (5.384615384615385)
Win/lose count 6.0/2.0. Average score (5.285714285714286)
Win/lose count 6.0/0. Average score (5.333333333333333)
Win/lose count 4.5/3.0. Average score (5.09375)
Win/lose count 3.0/0. Average score (4.970588235294118)
Win/lose count 3.5/1.0. Average score (4.833333333333333)
Win/lose count 3.5/0. Average score (4.7631578947368425)
Win/lose

The CNN models performs better  than the FC's.

When we increase the temperature, results improve for both models because we are increasing the probability of having cheese and lowering the probability of poisonious cells

But in both models, the agent stops exploring the island.



***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [38]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        

def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state,train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        self.malus_position = np.zeros((grid_size,grid_size))      
        
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
       
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1 
        reward = reward + self.board[self.x, self.y]
       
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), 
                                self.board.reshape(self.grid_size, self.grid_size,1), 
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1), 
                                        self.board.reshape(self.grid_size, self.grid_size,1), 
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [39]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 001/030 | Loss 0.0185 | Win/lose count 5.0/23.30000000000006 (-18.30000000000006)
Epoch 002/030 | Loss 0.0066 | Win/lose count 4.5/27.20000000000008 (-22.70000000000008)
Epoch 003/030 | Loss 0.0335 | Win/lose count 8.900000000000002/29.300000000000125 (-20.400000000000123)
Epoch 004/030 | Loss 0.0091 | Win/lose count 2.9/24.100000000000065 (-21.200000000000067)
Epoch 005/030 | Loss 0.0264 | Win/lose count 5.800000000000001/35.100000000000136 (-29.300000000000136)
Epoch 006/030 | Loss 0.0067 | Win/lose count 6.900000000000002/32.40000000000007 (-25.500000000000068)
Epoch 007/030 | Loss 0.0071 | Win/lose count 3.1999999999999997/29.00000000000015 (-25.80000000000015)
Epoch 008/030 | Loss 0.0086 | Win/lose count 3.6999999999999997/34.2000000000001 (-30.500000000000103)
Epoch 009/030 | Loss 0.0067 | Win/lose count 6.400000000000001/35.20000000000014 (-28.800000000000136)
Epoch 010/030 | Loss 0.0093 | Win/lose count 3.9999999999999996/35.500000000000114 (-31.500000000000114)
Epoch 011

In [40]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 1.2000000000000002/21.800000000000047. Average score (-20.600000000000048)
Win/lose count 3.9999999999999996/25.1000000000001. Average score (-20.850000000000072)
Win/lose count 5.2/24.80000000000006. Average score (-20.4333333333334)
Win/lose count 2.8/25.400000000000066. Average score (-20.97500000000007)
Win/lose count 4.3999999999999995/40.00000000000017. Average score (-23.900000000000087)
Win/lose count 1.6/21.70000000000001. Average score (-23.26666666666674)
Win/lose count 3.9999999999999996/31.10000000000014. Average score (-23.8142857142858)
Win/lose count 1.6/25.700000000000102. Average score (-23.850000000000087)
Win/lose count 7.200000000000002/26.30000000000007. Average score (-23.322222222222308)
Win/lose count 2.8/25.4000000000001. Average score (-23.250000000000085)
Win/lose count 5.2/26.80000000000008. Average score (-23.100000000000083)
Win/lose count 5.2/21.800000000000047. Average score (-22.55833333333342)
Win/lose count 7.200000000000002/23.3000000

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***